In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [4]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, accuracy_score
from statsmodels.tsa.arima.model import ARIMA

In [5]:
AMZN = pd.read_csv("/kaggle/input/price-volume-data-for-all-us-stocks-etfs/Stocks/aapl.us.txt")

In [6]:
AMZN['Date'] = pd.to_datetime(AMZN['Date'])
AMZN.set_index('Date', inplace=True)
AMZN.sort_index(inplace=True)

In [7]:
last_date_in_AMZN =AMZN.index.get_level_values('Date').max()
print(last_date_in_AMZN)

2017-11-10 00:00:00


In [8]:
most_recent_date_in_AMZN = AMZN.index.get_level_values('Date').max()
ten_years_prior_to_recent = most_recent_date_in_AMZN - pd.DateOffset(years=10)

AMZN = AMZN[AMZN.index.get_level_values('Date') >= ten_years_prior_to_recent]

In [9]:
last_date_in_AMZN = AMZN.index.get_level_values('Date').min()
print(last_date_in_AMZN)

2007-11-12 00:00:00


In [10]:
def df_transform(df):
    df["Daily Return"] = df["Close"].pct_change()

    df["7d_MA"] = df["Close"].transform(lambda x: x.rolling(window=7).mean())

    df["30d_MA"] = df["Close"].transform(lambda x: x.rolling(window=30).mean())

    df["Roll_vol"] = df["Daily Return"].transform(lambda x: x.rolling(window=30).std())

    return df

In [11]:
AMZN_new = df_transform(AMZN)

In [12]:
AMZN.head(10)

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,Open,High,Low,Close,Volume,OpenInt,Daily Return,7d_MA,30d_MA,Roll_vol
Date,,,,,,,,,,
2007-11-12,21.130,21.479,19.291,19.691,492362604,0,NaN,NaN,NaN,NaN
2007-11-13,20.615,21.897,19.691,21.765,484373501,0,0.105327,NaN,NaN,NaN
2007-11-14,22.733,22.739,20.970,21.274,403585172,0,-0.022559,NaN,NaN,NaN
2007-11-15,21.280,21.717,20.528,21.040,414487458,0,-0.010999,NaN,NaN,NaN
2007-11-16,21.193,21.388,20.405,21.309,385660112,0,0.012785,NaN,NaN,NaN
2007-11-19,21.262,21.540,20.758,20.996,321694225,0,-0.014689,NaN,NaN,NaN
2007-11-20,21.247,22.000,20.942,21.623,429930531,0,0.029863,21.099714,NaN,NaN
2007-11-21,21.195,22.072,21.086,21.572,339509885,0,-0.002359,21.368429,NaN,NaN
2007-11-23,22.027,22.033,21.739,21.967,129891462,0,0.018311,21.397286,NaN,NaN


In [ ]:
close_prices = AMZN['Close']
n = len(close_prices)
split_point = int(n * 0.8)
def create_lag_features(series, lags=5):
    df = pd.DataFrame()
    for i in range(1, lags+1):
        df[f'lag_{i}'] = series.shift(i)
    df['target'] = series
    return df.dropna()


In [22]:
def compute_indicators(df):
    df = df.copy()
    df['MA7'] = df['Close'].rolling(window=7).mean()
    df['MA30'] = df['Close'].rolling(window=30).mean()
    delta = df['Close'].diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.rolling(14).mean()
    avg_loss = loss.rolling(14).mean()
    rs = avg_gain / avg_loss
    df['RSI'] = 100 - (100 / (1 + rs))
    df = df.dropna()
    return df

features_df = compute_indicators(AMZN)
features_df = features_df.dropna()
X = features_df[['MA7', 'MA30', 'RSI']]
y = features_df['Close']

X_train = X.iloc[:split_point - (len(AMZN) - len(features_df))]
X_test = X.iloc[split_point - (len(AMZN) - len(features_df)):]
y_train = y.iloc[:split_point - (len(AMZN) - len(features_df))]
y_test = y.iloc[split_point - (len(AMZN) - len(features_df)):]

rf_model = RandomForestRegressor()
rf_model.fit(X_train, y_train)
pred_rf = rf_model.predict(X_test)
mae_rf = mean_absolute_error(y_test, pred_rf)
print(mae_rf)

/usr/local/lib/python3.11/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in greater
  return op(a, b)
/usr/local/lib/python3.11/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in less
  return op(a, b)


10.975940218253987


In [24]:
# Use ARIMA(5,1,0) as a start — you can tune (p,d,q) later
close_prices = AMZN['Close']
split_point = int(len(close_prices) * 0.8)
train = close_prices.iloc[:split_point]
test = close_prices.iloc[split_point:]
arima_model = ARIMA(train, order=(5,1,0))
arima_result = arima_model.fit()
pred_arima = arima_result.forecast(steps=len(test))
mae_arima = mean_absolute_error(test, pred_arima)
print(mae_arima)

/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


20.31170129725727


/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/base/tsa_model.py:837: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


In [26]:
!pip install backtesting

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.4/191.4 kB 9.2 MB/s eta 0:00:00


In [27]:
from backtesting import Backtest, Strategy
import pandas as pd
import numpy as np

class MLBasedStrategy(Strategy):
    def init(self):
        # Save the trained model
        self.model = self.__class__.model  # Access from static variable

    def next(self):
        # Get last 30 bars (to compute indicators)
        if len(self.data.Close) < 30:
            return  # not enough data yet

        close = pd.Series(self.data.Close[-31:])  # need one extra for diff

        # Compute features
        ma7 = close.rolling(window=7).mean().iloc[-1]
        ma30 = close.rolling(window=30).mean().iloc[-1]
        
        delta = close.diff()
        gain = delta.where(delta > 0, 0).rolling(14).mean().iloc[-1]
        loss = -delta.where(delta < 0, 0).rolling(14).mean().iloc[-1]
        rs = gain / loss if loss != 0 else 0
        rsi = 100 - (100 / (1 + rs)) if loss != 0 else 100

        # Prepare feature vector
        X_input = pd.DataFrame([[ma7, ma30, rsi]], columns=['MA7', 'MA30', 'RSI'])

        # Predict the next day's closing price
        predicted_price = self.model.predict(X_input)[0]
        current_price = self.data.Close[-1]

        # Simple signal logic
        if predicted_price > current_price and not self.position:
            self.buy()
        elif predicted_price < current_price and self.position:
            self.sell()

/usr/local/lib/python3.11/dist-packages/backtesting/_plotting.py:55: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support, such as old IDEs. Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.seasonal import seasonal_decompose

# --------- Load and preprocess ---------
df = pd.read_csv("your_data.csv", parse_dates=['Date'], index_col='Date')  # adjust filename
df['log_close'] = np.log(df['Close'])

# Example feature: RSI × Volume
def compute_rsi(series, period=14):
    delta = series.diff()
    gain = (delta.where(delta > 0, 0)).rolling(period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(period).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

df['RSI'] = compute_rsi(df['Close'])
df['RSI_Volume'] = df['RSI'] * df['Volume']

# More features
df['SMA_14'] = df['Close'].rolling(14).mean()
df['SMA_14_sq'] = df['SMA_14'] ** 2
df = df.dropna()

# --------- Lag Features ---------
def create_lag_features(series, lags=5):
    df_lag = pd.DataFrame()
    for i in range(1, lags+1):
        df_lag[f'lag_{i}'] = series.shift(i)
    df_lag['target'] = series
    return df_lag.dropna()

lr_data = create_lag_features(df['log_close'])
exog_vars = df[['RSI_Volume', 'SMA_14_sq']].fillna(method='bfill')
exog_vars = exog_vars.loc[lr_data.index]  # align

# --------- Train-Test Split ---------
split_point = int(len(lr_data) * 0.8)

train_lr = lr_data.iloc[:split_point]
test_lr = lr_data.iloc[split_point:]

# --------- Linear Regression ---------
lr_model = LinearRegression()
lr_model.fit(train_lr.drop('target', axis=1), train_lr['target'])
pred_lr = lr_model.predict(test_lr.drop('target', axis=1))
mae_lr = mean_absolute_error(test_lr['target'], pred_lr)

# --------- ARIMA ---------
train_arima = df['log_close'].iloc[:split_point]
test_arima = df['log_close'].iloc[split_point:]

arima_model = ARIMA(train_arima, order=(5,1,0))
arima_result = arima_model.fit()
pred_arima = arima_result.forecast(steps=len(test_arima))
mae_arima = mean_absolute_error(test_arima, pred_arima)

# --------- SARIMAX ---------
train_y = df['log_close'].iloc[:split_point]
test_y = df['log_close'].iloc[split_point:]
train_exog = exog_vars.iloc[:split_point]
test_exog = exog_vars.iloc[split_point:]

sarimax_model = SARIMAX(train_y, exog=train_exog, order=(5,1,0), seasonal_order=(1,1,1,12))
sarimax_result = sarimax_model.fit(disp=False)
pred_sarimax = sarimax_result.forecast(steps=len(test_y), exog=test_exog)
mae_sarimax = mean_absolute_error(test_y, pred_sarimax)

# --------- Random Forest ---------
rf_data = lr_data.merge(exog_vars, left_index=True, right_index=True)
X_rf = rf_data.drop(columns=['target'])
y_rf = rf_data['target']
X_rf_train = X_rf.iloc[:split_point]
y_rf_train = y_rf.iloc[:split_point]
X_rf_test = X_rf.iloc[split_point:]
y_rf_test = y_rf.iloc[split_point:]

rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_rf_train, y_rf_train)
pred_rf = rf_model.predict(X_rf_test)
mae_rf = mean_absolute_error(y_rf_test, pred_rf)

# --------- Ensemble Model ---------
ensemble_preds = (
    pred_lr * (1/mae_lr) +
    pred_arima.values * (1/mae_arima) +
    pred_rf * (1/mae_rf)
)
ensemble_preds /= (1/mae_lr + 1/mae_arima + 1/mae_rf)

mae_ensemble = mean_absolute_error(test_lr['target'], ensemble_preds)

# --------- Directional Accuracy ---------
def directional_accuracy(y_true, y_pred):
    return np.mean(np.sign(np.diff(y_true)) == np.sign(np.diff(y_pred)))

da_sarimax = directional_accuracy(test_y.values, pred_sarimax.values)
da_ensemble = directional_accuracy(test_lr['target'].values, ensemble_preds)

# --------- Output Summary ---------
print(f"MAE - Linear Regression: {mae_lr:.4f}")
print(f"MAE - ARIMA: {mae_arima:.4f}")
print(f"MAE - SARIMAX: {mae_sarimax:.4f}")
print(f"MAE - Random Forest: {mae_rf:.4f}")
print(f"MAE - Ensemble: {mae_ensemble:.4f}")
print()
print(f"Directional Accuracy - SARIMAX: {da_sarimax:.2%}")
print(f"Directional Accuracy - Ensemble: {da_ensemble:.2%}")

# --------- Feature Importance Plot ---------
feature_imp = pd.Series(rf_model.feature_importances_, index=X_rf.columns)
sns.barplot(x=feature_imp, y=feature_imp.index)
plt.title("Random Forest Feature Importances")
plt.show()
